In [1]:
!pip install binance_historical_data mplfinance stable-baselines3 backtesting tensorboard binance ta scikit-learn
# !pip install line_profiler


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
#!git clone https://github.com/Klok-e/python-price-predict-experiment.git

import sys

sys.path.insert(0, 'python-price-predict-experiment')

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
# 
# import time
# import os
# try:
#   os.makedirs("./drive/MyDrive/colab/computed-data/")
# except:
#   pass

In [4]:
from util import download_and_process_data_if_available

df_tickers = download_and_process_data_if_available("cache/df_tickers.pkl")

Loading data from cache


In [5]:
%load_ext tensorboard
%tensorboard --logdir data/tensorboard
# %load_ext tensorboard
# %tensorboard --logdir /content/drive/MyDrive/colab/computed-data/tensorboard

Launching TensorBoard...

In [6]:
from train_model import train_model
from env import CustomEnv

from stable_baselines3.common.env_checker import check_env

env = CustomEnv(df_tickers)
check_env(env)
del env

hidden_size = 512
lstm_layers = 2
arch = [256, 256, 256]
window_size = 128
print(f"hidden_size {hidden_size}, lstm_layers {lstm_layers}, window_size {window_size}")
# dir = "/content/drive/MyDrive/colab/computed-data"
dir = "data"
train_model(df_tickers, hidden_size, lstm_layers, arch, 10_000, window_size, 2, dir)

obs length = {'prices_sequence': (64, 12), 'other': (2,)}
Initialized with seed 0
hidden_size 512, lstm_layers 2, window_size 128


/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:251: UserWarning: Your observation prices_sequence has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


Using cuda device
Logging to data/tensorboard/PPO_6
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 558      |
|    ep_rew_mean     | -2.09    |
| time/              |          |
|    fps             | 102      |
|    iterations      | 1        |
|    time_elapsed    | 40       |
|    total_timesteps | 4096     |
---------------------------------


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.75 GiB. GPU 0 has a total capacty of 3.94 GiB of which 1.18 GiB is free. Process 106711 has 148.00 MiB memory in use. Including non-PyTorch memory, this process has 1.44 GiB memory in use. Of the allocated memory 1.35 GiB is allocated by PyTorch, and 21.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import numpy as np
from util import load_pickle
import os
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
from trading_metrics import calculate_metrics
import re
import pandas as pd


def plot_equity(model_name, model_equities, ax1, ax2, plot_hist):
    match = re.search("ws(\d+)", model_name)
    in_obs = int(match.group(1)) if match else 64
    skip_steps = 1024 + in_obs

    sum_equity = None
    buy_trades = []
    sell_trades = []
    for item in model_equities:
        if sum_equity is None:
            sum_equity = item[2]["Equity"].iloc[skip_steps:].copy()
        else:
            sum_equity += item[2]["Equity"].iloc[skip_steps:]

        if len(item[1]) > 0:
            buy_trades.append(item[1]["EntryTime"])
            sell_trades.append(item[1]["ExitTime"])

    if len(buy_trades) > 0:
        buy_trades = pd.concat(buy_trades)
        sell_trades = pd.concat(sell_trades)

        combined_trades = pd.DataFrame({
            "Buy trades": buy_trades,
            "Sell trades": sell_trades
        })

    start_cash = 1_000_000
    tickers_count = 5
    trades_count = sum(map(lambda x: len(x[1]), model_equities))
    metrics = calculate_metrics(sum_equity, trades_count, start_cash * tickers_count)

    sum_equity = sum_equity.rolling(window=256).mean()[256:]

    y = (sum_equity - start_cash * tickers_count) / (start_cash * tickers_count)

    # Plot line on the primary axis
    ax1.plot(y, markevery=1024, label=model_name)

    # Plot histogram on the secondary axis
    if plot_hist and len(buy_trades) > 0:
        ax2.hist(combined_trades, bins=32, alpha=0.5, label=["Buy trades", "Sell trades"], color=['green', 'red'])

    print()
    print(f"{model_name} metrics:")
    print(f"cumulative_return={metrics[0]:.4f}, "
          f"max_earning_rate={metrics[1]:.4f}, "
          f"maximum_pullback={metrics[2]:.4f}, "
          f"average_profitability_per_trade={metrics[3]:.4f}, "
          f"sharpe_ratio={metrics[4]:.4f}")
    print()


filenames = next(os.walk(f"backtest-results/"), (None, None, []))[2]  # [] if no file
results = {}
for filename in filenames:
    (trades, equity_curve) = load_pickle(f"backtest-results/{filename}")
    parts = filename.split("_")
    model_name = "_".join(parts[:-1])  # Get all parts except the last one as the model name
    coin = parts[-1].split('.')[0]  # Get the last part as the coin, and remove file extension

    if model_name not in results:
        results[model_name] = []
    results[model_name].append((coin, trades, equity_curve, filename))

buy_hold = "Buy and Hold"
buy_hold_res = results.pop(buy_hold)

plt.figure(figsize=(10, 7))

ax1 = plt.gca()
ax2 = ax1.twinx()

for i, model in enumerate(sorted(list(results.keys()))):
    plot_equity(model, results[model], ax1, ax2, True)

    plot_equity(buy_hold, buy_hold_res, ax1, ax2, False)

    ax2.set_ylim([0, max(ax2.get_ylim()[1], ax1.get_ylim()[1])])

    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax1.xaxis.set_major_locator(mdates.AutoDateLocator())
    plt.gcf().autofmt_xdate()
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")
    plt.tight_layout()

    # plt.show()
    plt.savefig(f"pics/figure{i}.png")
    ax1.cla()
    ax2.cla()

In [ ]:
list(results.items())[0][1][0][1]